## 数据处理

### 总亮度

标准化方法：
- 假设服从平方反比，标准到 $1AU$ 的结果
- 所有数据计算得到平均值 $m$ ，标准差 $sigma$
- 标准化


In [ ]:
import astropy.units as u
import sunpy.map
import numpy as np
import os
import h5py
import matplotlib.pyplot as plt
from pathlib import Path
from concurrent.futures import ProcessPoolExecutor
from glob import glob
import logging
from datetime import datetime
import gc  # Add garbage collection
import time  # For timing stats

# Configure logging
log_dir = "Data-process"
log_file = Path(log_dir) / "processing.log"
logging.basicConfig(filename=log_file, level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

def normalize_distance(data_map):
    """Normalize the data based on distance to Sun in AU."""
    data_out = data_map.data * (data_map.dsun.to(u.AU)/(1 * u.AU).value)**2
    return data_out

def save_as_hdf5(data, output_path):
    """Save normalized data as HDF5 file."""
    with h5py.File(output_path, 'w') as hf:
        hf.create_dataset('data', data=data, compression="gzip", compression_opts=9)

def is_processed(input_path, output_dir):
    """Check if a file has already been processed."""
    base_name = Path(input_path).stem
    hdf5_path = Path(output_dir) / f"{base_name}_nd.h5"
    return hdf5_path.exists()

def get_output_path(input_path, output_dir):
    """Get the output HDF5 path for a given input file."""
    base_name = Path(input_path).stem
    return Path(output_dir) / f"{base_name}_nd.h5"

def process_fits_file(input_path, output_dir, allow_errors=False):
    """Process a FITS file and save normalized data as HDF5."""
    try:
        # Load and process data
        data_map = sunpy.map.Map(input_path)
        normalized_data = normalize_distance(data_map)
        
        # Generate output filename and save
        hdf5_path = get_output_path(input_path, output_dir)
        save_as_hdf5(normalized_data, hdf5_path)
        
        # Explicitly clear references to free memory
        del data_map
        del normalized_data
        gc.collect()  # Force garbage collection
        
        # logging.info(f"Saved HDF5 file: {hdf5_path}")
        
        return True
        
    except Exception as e:
        logging.error(f"Error processing file {input_path}: {e}")
        if not allow_errors:
            raise
        return False


if __name__ == "__main__":
    start_time = time.time()
    os.chdir(r"E:\study-and-research\sunspot-sith-sora")

    # Get all FITS files in the input directory
    input_dir = Path(r"data\origin\Ic_720s")
    output_dir = Path(r"data\processed\Ic_720s_normalize_distance")
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Pre-filter files to process
    fits_files = list(input_dir.glob("*.fits"))
    total_files = len(fits_files)
    
    # Pre-check which files need processing
    files_to_process = []
    skipped_files = 0
    
    logging.info(f"Checking {total_files} files for processing status...")
    
    for file in fits_files:
        if is_processed(file, output_dir):
            skipped_files += 1
        else:
            files_to_process.append(file)
    
    need_processing = len(files_to_process)
    logging.info(f"Found {need_processing} files to process, {skipped_files} already processed")
    
    # Counter for processed files
    processed_files = 0
    error_files = 0

    # Process files in parallel with optimal workers
    max_workers = 10
    
    if files_to_process:
        logging.info(f"Starting processing with {max_workers} workers")
        with ProcessPoolExecutor(max_workers=max_workers) as executor:
            # Process files in batches to prevent memory buildup
            batch_size = 10
            for i in range(0, len(files_to_process), batch_size):
                batch = files_to_process[i:i+batch_size]
                # logging.info(f"Processing batch {i//batch_size + 1} of {(len(files_to_process)-1)//batch_size + 1} ({len(batch)} files)")
                
                # Submit batch for processing
                futures = [
                    executor.submit(process_fits_file, str(file), output_dir, True)
                    for file in batch
                ]
                
                # Process results
                for future in futures:
                    try:
                        result = future.result()
                        if result is True:
                            processed_files += 1
                        else:
                            error_files += 1
                    except Exception as e:
                        error_files += 1
                        logging.error(f"Failed to process a file: {e}")
                
                # Ensure memory is freed between batches
                gc.collect()

    end_time = time.time()
    elapsed_time = end_time - start_time
    
    logging.info(f"Processing complete in {elapsed_time:.2f} seconds:")
    logging.info(f"Total files: {total_files}")
    logging.info(f"Already processed (skipped): {skipped_files}")
    logging.info(f"Successfully processed: {processed_files}")
    logging.info(f"Failed to process: {error_files}")
    
    if processed_files > 0:
        logging.info(f"Average processing time per file: {elapsed_time/processed_files:.2f} seconds")

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import time
import datetime
import concurrent.futures
from tqdm.notebook import tqdm
import os
import pandas as pd
import re

os.chdir(r"E:\study-and-research\sunspot-with-sora")

# Directory containing the processed h5 files
data_dir = Path("data/processed/Ic_720s_normalize_distance")

# Get all h5 files
h5_files = list(data_dir.glob("*.h5"))
print(f"Found {len(h5_files)} h5 files")

def extract_timestamp(filename):
    """
    Extract timestamp from the filename.
    Assumes filename format contains date/time information.
    Modify the regex pattern based on your actual filename format.
    """
    # Example pattern: looking for something like YYYY-MM-DD_HH-MM-SS
    pattern = r'(\d{4}-\d{2}-\d{2}[_T]\d{2}[-:]\d{2}[-:]\d{2})'
    match = re.search(pattern, filename)
    
    if match:
        # Convert matched string to datetime
        dt_str = match.group(1).replace('_', 'T').replace('-', ':')
        return datetime.datetime.fromisoformat(dt_str)
    
    # Fallback: use file modification time if pattern doesn't match
    file_stat = os.stat(filename)
    return datetime.datetime.fromtimestamp(file_stat.st_mtime)

def process_file(file_path):
    """Process a single file to extract timestamp and calculate brightness."""
    try:
        with h5py.File(file_path, 'r') as hf:
            # Get data and calculate brightness
            data = hf['data'][:]
            brightness = np.nansum(data)
            
            # Try to get timestamp from file metadata if available
            timestamp = None
            if 'timestamp' in hf.attrs:
                timestamp = hf.attrs['timestamp']
            else:
                # Extract from filename
                timestamp = extract_timestamp(file_path)
                
            return timestamp, brightness, file_path.name
    except Exception as e:
        print(f"Error processing {file_path.name}: {e}")
        return None

# Calculate total brightness for each file in parallel
print("Calculating total brightness for each file in parallel...")
start_time = time.time()

# Define number of workers based on CPU cores
num_workers = os.cpu_count() - 1  # Leave one core free
if num_workers < 1:
    num_workers = 1

results = []
with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor:
    # Submit all tasks
    futures = {executor.submit(process_file, file_path): file_path for file_path in h5_files}
    
    # Process results as they complete
    for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
        file_path = futures[future]
        try:
            result = future.result()
            if result is not None:
                results.append(result)
        except Exception as e:
            print(f"Error processing {file_path}: {e}")

end_time = time.time()
print(f"Parallel processing completed in {end_time - start_time:.2f} seconds")

# Convert results to a DataFrame for easier manipulation
brightness_df = pd.DataFrame(results, columns=['timestamp', 'brightness', 'filename'])
brightness_df = brightness_df.sort_values('timestamp')

# Save to CSV
csv_path = "data/processed/total_brightness_timeseries.csv"
brightness_df.to_csv(csv_path, index=False)
print(f"Saved time series data to {csv_path}")

# Also save to numpy format
np_path = "data/processed/total_brightness_timeseries.npz"
np.savez(np_path, 
         timestamps=brightness_df['timestamp'].astype(np.datetime64).values,
         brightness=brightness_df['brightness'].values,
         filenames=brightness_df['filename'].values)
print(f"Saved numpy array to {np_path}")

# Display basic statistics
brightness_array = brightness_df['brightness'].values

print("\nBasic Statistics:")
print(f"Total number of files processed: {len(brightness_df)}")
print(f"Mean total brightness: {np.mean(brightness_array):.2e}")
print(f"Standard deviation: {np.std(brightness_array):.2e}")
print(f"Min: {np.min(brightness_array):.2e}")
print(f"Max: {np.max(brightness_array):.2e}")

# Plot time series data
plt.figure(figsize=(14, 6))
plt.plot(brightness_df['timestamp'], brightness_df['brightness'])
plt.title('Total Brightness Over Time')
plt.xlabel('Time')
plt.ylabel('Total Brightness')
plt.grid(True)
plt.tight_layout()
plt.savefig('data/processed/brightness_timeseries.png')
plt.show()

# Continue with the sampling stability analysis
# (keep your existing code for this part)
sample_sizes = [10, 50, 100, 200, 500, 1000, 2000, 5000]
if len(brightness_df) < 5000:
    # Adjust sample sizes if we have fewer data points
    sample_sizes = [s for s in sample_sizes if s <= len(brightness_df)]
    if len(sample_sizes) == 0:
        sample_sizes = [1, 5, 10, len(brightness_df)//2, len(brightness_df)]

num_experiments = 100  # Number of random sampling experiments for each size
sampling_results = []

for sample_size in tqdm(sample_sizes):
    means = []
    stds = []
    
    for _ in range(num_experiments):
        if sample_size < len(brightness_array):
            sample = np.random.choice(brightness_array, size=sample_size, replace=False)
        else:
            sample = brightness_array
            
        means.append(np.mean(sample))
        stds.append(np.std(sample))
    
    sampling_results.append({
        'sample_size': sample_size,
        'mean_of_means': np.mean(means),
        'std_of_means': np.std(means),
        'mean_of_stds': np.mean(stds)
    })

# Rest of your plotting code as before

### 图像

标准化方法：
- 随机化计算有效数据点数的平均值 $n$
- 计算标准化系数：$m_f=1$ , $sigma_f=sigma/m$ （考虑到 nolimbdark 数据为相对值，用 m 代替平静太阳的值）
- 选取 $4 sigma$ 内结果映射到 $[0,255]$

另一种处理方法，采用映射 $e^{-x+1}$ 在 1 附近近似线性

In [ ]:
import astropy.units as u
import sunpy.map
import numpy as np
import matplotlib.pyplot as plt

%cd "E:/study-and-research/sunspot-with-sora"
data_in = sunpy.map.Map(
    r"data\origin\flare_Ic_nolimbdark_720s\hmi.ic_nolimbdark_720s.20240222_200000_TAI.3.continuum.fits"
)
data_in2 = sunpy.map.Map(r"data\origin\Ic_720s\hmi.ic_720s.20200601_000000_TAI.3.continuum.fits")

np.nanmin(data_in.data)

data_in2.meta

### TODO: 验证 DN/s 是否服从平方反比

## Training

### downsample image

In [ ]:
from PIL import Image

%cd "E:\study-and-research\sunspot-with-sora"
# Load image
image = Image.open(r"data\processed\figure\figure-origin\hmi.ic_nolimbdark_720s.20211101_000000_TAI.3.continuum_nd.png")

# Resize using Lanczos
resized_image = image.resize((960, 960), Image.LANCZOS)

# Save or show the result
resized_image.save(r"data\processed\figure\figure-downsample\hmi_ic_resized.png")


### Train VAE

#### Construct dataset

采用滑动窗口形式构造 dataset

TODO 实验不同窗口大小与窗口间隔的影响

初步采用：
窗口大小：16（~1/4太阳周）
间隔：12

## Test

In [ ]:
import astropy.units as u
import sunpy.map

import numpy as np

import matplotlib.pyplot as plt

%cd "E:\study-and-research\sunspot-with-sora"
data_in = sunpy.map.Map(
    "hmi.ic_720s.2024-02-20T00_10_32.3Z.838005.continuum.fits"
)


In [ ]:
import matplotlib.pyplot as plt
from utils.fits_tools import fix_and_load_fits

%cd "E:\study-and-research\sunspot-with-sora"
# Load the FITS file with fixed metadata
data_in = fix_and_load_fits("Data-process/hmi.ic_720s.6173.831698.2024-01-01T12_10_29.700Z.continuum.fits")

# Now you can work with the data_in SunPy map
plt.figure(figsize=(10, 10))
data_in.plot()
plt.colorbar()
plt.show()

In [ ]:
from astropy.io import fits
import sunpy.map

fits_file = "Data-process\hmi.ic_720s.6173.831698.2024-01-01T12_10_29.700Z.continuum.fits"
data_in = sunpy.map.Map(fits_file, {**fits.getheader(fits_file), 'CUNIT1': 'arcsec', 'CUNIT2': 'arcsec'})

In [ ]:
data_in.data.shape

In [ ]:
data_in.meta.get('BUNIT')


In [4]:
import numpy as np
data2 = np.load(r'E:\study-and-research\sunspot-with-sora\dataset\training\brightness\L16-S8\20211101_000000.npz')

data2['data']


FileNotFoundError: [Errno 2] No such file or directory: 'E:\\study-and-research\\sunspot-with-sora\\dataset\\training\\brightness\\L16-S8\\20211101_000000.npz'

In [6]:
import numpy as np
# Load the brightness time series data
data = np.load(r'E:\study-and-research\sunspot-with-sora\data\processed\brightness\filtered\filtered_brightness_timeseries.npz')




In [ ]:
import numpy as np
# Load the brightness time series data
data = np.load(r'E:\study-and-research\sunspot-with-sora\data\processed\brightness\Ic_720s_dn_brightness_stats.npz')

print(data['mean'])
print(data['std'])